Importing Required Libraries

In [7]:
# Data Manipulation and Analysis
import pandas as pd
import numpy as np

# Machine Learning and Preprocessing
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import silhouette_score, mean_absolute_error, mean_squared_error, r2_score, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split

# Deep Learning Framework
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Visualization Libraries
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
# Load the dataset
file_path = "/content/Mera_CleanData.csv"
data = pd.read_csv(file_path)
data = data.drop('extreme_weather', axis=1)

In [9]:
data

,date,max_temp,min_temp,igmin,min_grass,rain,pressure_cbl,wind_speed,max_10minute_wind,dir_10minute_wind,...,day,temp_range,smd_combined,wind_category,extreme_rain,heatwave,high_wind,extreme_weather_combination,sun_category,season
0,1979-10-28,-0.27,-0.36,1,-1.18,0.9,997.9,-0.31,-0.26,0.89,...,28,7.5,-0.13,W,0,1,1,0-1-1,Extremely High,Autumn
1,1979-10-29,-0.48,-0.48,0,-0.29,0.7,998.8,-0.45,-0.88,0.42,...,29,7.0,-0.13,SW,0,1,1,0-1-1,Moderate,Autumn
2,1979-10-30,0.48,0.60,0,0.62,4.8,990.8,0.92,0.96,0.07,...,30,6.9,-2.30,SW,1,1,1,1-1-1,Moderate,Autumn
3,1979-10-31,-0.15,0.22,0,0.21,0.5,990.0,0.74,0.96,0.54,...,31,5.5,-0.82,W,0,1,1,0-1-1,Extremely High,Autumn
4,1979-11-01,-0.40,-0.14,0,-0.04,0.0,1001.6,0.51,0.20,0.42,...,1,5.9,0.30,SW,0,1,1,0-1-1,Extremely High,Autumn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16436,2024-10-27,0.26,-0.50,0,-0.50,3.1,1007.8,-0.27,-0.42,-0.04,...,27,10.7,2.32,S,1,1,1,1-1-1,Moderate,Autumn
16437,2024-10-28,0.28,1.15,0,0.64,1.1,1007.8,-0.36,-0.88,0.77,...,28,3.5,1.92,W,0,1,1,0-1-1,Moderate,Autumn
16438,2024-10-29,-0.38,0.76,0,0.85,0.0,1015.1,-0.70,-0.88,1.01,...,29,2.0,2.25,W,0,1,1,0-1-1,Moderate,Autumn
16439,2024-10-30,0.34,0.78,0,0.75,0.0,1018.7,-1.34,-1.49,0.89,...,30,5.4,2.65,W,0,1,1,0-1-1,Extremely High,Autumn


Cluster Analysis

In [10]:
# Select relevant features for clustering
features = ['extreme_rain', 'heatwave', 'high_wind']
data_clustering = data[features].dropna()

# Standardize features for clustering
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data_clustering)

# Apply K-Means Clustering
kmeans = KMeans(n_clusters=3, random_state=42)  # Adjust n_clusters based on your requirements
kmeans_labels = kmeans.fit_predict(scaled_features)
silhouette_kmeans = silhouette_score(scaled_features, kmeans_labels)

# Add cluster labels to the dataset
data_clustering['Cluster'] = kmeans_labels

# Visualization: Extreme Rain vs High Wind
fig1 = px.scatter(
    data_clustering,
    x='extreme_rain',
    y='high_wind',
    color='Cluster',
    title="K-Means Clustering: Extreme Rain vs High Wind",
    labels={'Cluster': 'Cluster'},
    template="plotly_dark"
)
fig1.update_layout(
    xaxis_title="Extreme Rain",
    yaxis_title="High Wind",
    legend_title="Cluster",
    height=600,
    width=800
)
fig1.show()

print("\n")

#Visualization: Heatwave vs High Wind
fig2 = px.scatter(
    data_clustering,
    x='heatwave',
    y='high_wind',
    color='Cluster',
    title="K-Means Clustering: Heatwave vs High Wind",
    labels={'Cluster': 'Cluster'},
    template="plotly_dark"
)
fig2.update_layout(
    xaxis_title="Heatwave",
    yaxis_title="High Wind",
    legend_title="Cluster",
    height=600,
    width=800
)
fig2.show()

print("\n")

# 3D Scatter Plot for Extreme Rain, Heatwave, and High Wind
fig3 = px.scatter_3d(
    data_clustering,
    x='extreme_rain',
    y='heatwave',
    z='high_wind',
    color='Cluster',
    title="K-Means Clustering: 3D Visualization",
    labels={'Cluster': 'Cluster'},
    template="plotly_dark"
)
fig3.update_layout(
    scene=dict(
        xaxis_title="Extreme Rain",
        yaxis_title="Heatwave",
        zaxis_title="High Wind"
    ),
    height=700,
    width=900
)
fig3.show()
print("\n")
# showing cluster averages for further insights
print(f"K-Means Silhouette Score: {silhouette_kmeans}")
cluster_means = data_clustering.groupby('Cluster').mean()
print("Cluster Means:\n", cluster_means)



K-Means Silhouette Score: 0.9902372861962644
Cluster Means:
          extreme_rain  heatwave  high_wind
Cluster                                   
0             0.00000       1.0   0.999501
1             1.00000       1.0   1.000000
2             0.21393       0.0   1.000000


LSTM Model

In [11]:
rain_weight = 0.333
heatwave_weight = 0.333
high_wind_weight = 0.333

data['Extreme_Weather'] = (
    data['extreme_rain'] * rain_weight +
    data['heatwave'] * heatwave_weight +
    data['high_wind'] * high_wind_weight
)
data['Extreme_Weather'] = data['Extreme_Weather'] / data['Extreme_Weather'].max()

# Define the features to include in the model
features = [
    'max_temp', 'min_temp', 'temp_range', 'rain', 'pressure_cbl', 'wind_speed',
    'max_10minute_wind', 'dir_10minute_wind', 'max_gust', 'sun', 'global_radiation',
    'soil', 'potential_evap', 'evap', 'smd_combined',
    'heatwave', 'high_wind'
]

target = 'Extreme_Weather'

# Handle missing or invalid values in features
data[features] = data[features].replace(r'^\s*$', np.nan, regex=True)  # Replace empty strings with NaN
data[features] = data[features].astype(float)  # Convert to numeric
data[features] = data[features].fillna(data[features].mean())  # Fill NaN with column mean

# Normalize features
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])

# Create sequences for LSTM
def create_sequences(data, target_col, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i + seq_length][features].values)
        y.append(data.iloc[i + seq_length][target_col])
    return np.array(X), np.array(y)

seq_length = 90
X, y = create_sequences(data, target, seq_length)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Build LSTM Model
model = Sequential([
    LSTM(128, activation='relu', return_sequences=True, input_shape=(seq_length, len(features))),
    Dropout(0.3),
    LSTM(64, activation='relu'),
    Dense(1, activation='linear')  # Use linear activation for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
y_pred = model.predict(X_test).flatten()

# Metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Classification Metrics (using a threshold)
threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(int)
y_test_binary = (y_test > threshold).astype(int)

accuracy = accuracy_score(y_test_binary, y_pred_binary)
precision = precision_score(y_test_binary, y_pred_binary, zero_division=1)
f1 = f1_score(y_test_binary, y_pred_binary, zero_division=1)

# Plot training history
fig_history = go.Figure()
fig_history.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Training Loss'))
fig_history.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='Validation Loss'))
fig_history.update_layout(title="LSTM Training and Validation Loss",
                          xaxis_title="Epochs",
                          yaxis_title="Loss",
                          template="plotly_dark")
fig_history.show()

# Plot predictions vs actual
fig_predictions = go.Figure()
fig_predictions.add_trace(go.Scatter(y=y_test, mode='lines', name='Actual'))
fig_predictions.add_trace(go.Scatter(y=y_pred, mode='lines', name='Predicted'))
fig_predictions.update_layout(title="Predicted vs Actual Extreme Weather",
                               xaxis_title="Samples",
                               yaxis_title="Extreme Weather",
                               template="plotly_dark")
fig_predictions.show()

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



358/358 ━━━━━━━━━━━━━━━━━━━━ 17s 36ms/step - loss: 0.0409 - mae: 0.1503 - val_loss: 0.0221 - val_mae: 0.1229
Epoch 2/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - loss: 0.0231 - mae: 0.1237 - val_loss: 0.0236 - val_mae: 0.1045
Epoch 3/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - loss: 0.0222 - mae: 0.1201 - val_loss: 0.0217 - val_mae: 0.1214
Epoch 4/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - loss: 0.0215 - mae: 0.1206 - val_loss: 0.0215 - val_mae: 0.1207
Epoch 5/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - loss: 0.0223 - mae: 0.1224 - val_loss: 0.0215 - val_mae: 0.1215
Epoch 6/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - loss: 0.0217 - mae: 0.1205 - val_loss: 0.0222 - val_mae: 0.1109
Epoch 7/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - loss: 0.0213 - mae: 0.1181 - val_loss: 0.0212 - val_mae: 0.1251
Epoch 8/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - loss: 0.0215 - mae: 0.1211 - val_loss: 0.0213 - val_mae: 0.1278
Epoch 9/100
358/358 ━━━━━━━━━━━━━━━━━━━━ 10s

In [14]:
print(f"""
LSTM Model Performance:
------------------------
Accuracy: {accuracy:.4f}
Precision: {precision:.4f}
F1-Score: {f1:.4f}
Mean Absolute Error (MAE): {mae:.4f}
Root Mean Squared Error (RMSE): {mse:.4f}
R² Score: {r2:.4f}
""")


LSTM Model Performance:
------------------------
Accuracy: 0.9898
Precision: 0.9898
F1-Score: 0.9949
Mean Absolute Error (MAE): 0.1154
Root Mean Squared Error (RMSE): 0.0200
R² Score: 0.1293

